### Imports & useful functions

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
def fol_nb(text):
    fol = text.split()[0]
    return fol
def clean(row):
    return str(row).replace(",","").replace(" ","")
def date(date):
    date_new = datetime.datetime.strptime(date, "%B %d, %Y")
    return date_new
def date_inst(date):
    date_time_obj = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S+00:00')
    return date_time_obj

### Go over collected history, caluclate evolution 

In [3]:
gathered=[]
data_inst = pd.read_csv(r'C:\Users\basti\DataCamp\Projet_Groupe_vclean\data\3_clean_data\data_inst.csv')
for infl in data_inst[data_inst['influencer']!='Dua Lipa']['influencer'].unique():
    path = r"C:\Users\basti\DataCamp\Projet_Groupe\followers" 
    fol_c = pd.read_csv(path+'\\'+ infl.lower()+'.csv')
    fol_c = fol_c.drop(columns=['id','Following - (change)', 'Posts - (change)','Influence - (change)'])
    fol_c['nb_fol'] = fol_c['Followers - (change)'].apply(fol_nb)
    fol_c = fol_c.drop(columns=['Followers - (change)'])
    fol_c['nb_fol'] = fol_c['nb_fol'].apply(clean).astype(int)  
    fol_c['Date'] = fol_c['Date'].apply(date)
    fol_c['posting_date']=fol_c['Date']
    fol_c = fol_c.drop(columns=['Date'])
    fol_c['num_followers']=fol_c['nb_fol']
    fol_c = fol_c.drop(columns=['nb_fol'])
    fol_c['posting_date']=fol_c['posting_date'].dt.date
    fol_c = fol_c.drop_duplicates(subset='posting_date',ignore_index=True)
    data_inst_n = data_inst[data_inst['influencer']==infl]
    data_inst_n = data_inst_n[['posting_date','num_followers']].sort_values(by='posting_date', ascending=True,ignore_index=True)
    data_inst_n['posting_date'] = data_inst_n['posting_date'].apply(date_inst)
    data_inst_n['posting_date'] = data_inst_n['posting_date'].dt.date
    fol_c = fol_c.append(data_inst_n.iloc[-1])
    fol_c = fol_c.sort_values(by='posting_date', ascending=True,ignore_index=True)
    fol_c['posting_date'] = pd.to_datetime(fol_c['posting_date'])
    fol_c['diff_days'] = fol_c['posting_date'].diff()
    fol_c['diff_days'] = fol_c['diff_days'].apply(lambda x : x.days)
    growth_rate = [(fol_c['num_followers'][i+1]/fol_c['num_followers'][i])**(1/fol_c['diff_days'][i+1]) - 1 for i in range(len(fol_c)-1)] 
    growth_rate.insert(0, np.NaN) 
    fol_c['growth_rate'] = growth_rate
    panda = []
    for i in range(len(fol_c)-1):
        if fol_c['diff_days'][i+1] > 1 : 
            posting_date = [fol_c['posting_date'][i] + datetime.timedelta(days=j+1) for j in range(int(fol_c['diff_days'][i+1]) - 1)]
            num_followers = [fol_c['num_followers'][i] * (1 + fol_c['growth_rate'][i+1])**(j+1) for j in range(int(fol_c['diff_days'][i+1]) - 1)]
            df = pd.DataFrame({'posting_date': posting_date, 'num_followers': num_followers})
            panda.append(df)
    fol_new = pd.concat(panda, ignore_index=True)
    prep = fol_c[['posting_date','num_followers']]
    fol_end = pd.concat([prep,fol_new])
    fol_end = fol_end.sort_values(by='posting_date', ascending=True,ignore_index=True)
    days = (fol_end['posting_date'].iloc[-1] - fol_end['posting_date'].iloc[0])
    days = days.days
    growth_rate = (fol_end['num_followers'].iloc[-1]/fol_end['num_followers'].iloc[0])**(1/days) - 1
    data_inst_n['posting_date'] = pd.to_datetime(data_inst_n['posting_date'])
    data_inst_n = data_inst_n.sort_values(by='posting_date', ascending=True,ignore_index=True)
    fol_end = fol_end.append(data_inst_n.iloc[0])
    fol_end = fol_end.sort_values(by='posting_date', ascending=True,ignore_index=True)
    days_delta = (fol_end['posting_date'].iloc[1] - fol_end['posting_date'].iloc[0])
    days_delta = days_delta.days
    if days_delta > 1 : 
        posting_date = [fol_end['posting_date'][1] - datetime.timedelta(days=j+1) for j in range(days_delta)]
        num_followers = [fol_end['num_followers'][1] / (1 + growth_rate)**(j+1) for j in range(days_delta)]
        df = pd.DataFrame({'posting_date': posting_date, 'num_followers': num_followers})
    fol_final = pd.concat([fol_end,df],ignore_index=True)
    fol_final = fol_final.sort_values(by='posting_date', ascending=True,ignore_index=True)
    fol_final= fol_final.drop(fol_final.index[0]).reset_index()
    fol_final= fol_final.drop_duplicates(subset=['posting_date']).reset_index()
    final = data_inst_n.merge(fol_final, how='right', on='posting_date')
    final = final.drop(columns=['num_followers_x','index'])
    final.rename(columns = {'num_followers_y':'num_followers'}, inplace = True) 
    final['influencer'] = infl
    final = final.drop_duplicates(subset=['posting_date'])
    gathered.append(final)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars


### Dua Lipa case

In [4]:
dua = pd.read_csv(r'C:\Users\basti\DataCamp\Projet_Groupe\dualipa_othersite.csv')
dua = dua.drop(columns=['Unnamed: 1','Unnamed: 3','Following', 'Unnamed: 5','Uploads','Unnamed: 7'])
dua.rename(columns = {'Date':'posting_date', 'Followers':'num_followers'}, inplace = True) 
fol_c = dua.drop(15)
fol_c['posting_date'] = pd.to_datetime(fol_c['posting_date'])
fol_c['num_followers'] = fol_c['num_followers'].apply(clean).astype(int) 
data_inst = pd.read_csv(r'C:\Users\basti\DataCamp\Projet_Groupe_vclean\data\3_clean_data\data_inst.csv')
fol_c = fol_c.drop_duplicates(subset='posting_date',ignore_index=True)
data_inst_n = data_inst[data_inst['influencer']=='Dua Lipa']
data_inst_n = data_inst_n[['posting_date','num_followers']].sort_values(by='posting_date', ascending=True,ignore_index=True)
data_inst_n['posting_date'] = data_inst_n['posting_date'].apply(date_inst)
data_inst_n['posting_date'] = data_inst_n['posting_date'].dt.date
fol_c = fol_c.append(data_inst_n.iloc[-1])
fol_c = fol_c.sort_values(by='posting_date', ascending=True,ignore_index=True)
fol_c['posting_date'] = pd.to_datetime(fol_c['posting_date'])
fol_c['diff_days'] = fol_c['posting_date'].diff()
fol_c['diff_days'] = fol_c['diff_days'].apply(lambda x : x.days)
growth_rate = [(fol_c['num_followers'][i+1]/fol_c['num_followers'][i])**(1/fol_c['diff_days'][i+1]) - 1 for i in range(len(fol_c)-1)] 
growth_rate.insert(0, np.NaN) 
fol_c['growth_rate'] = growth_rate

panda = []

for i in range(len(fol_c)-1):
    if fol_c['diff_days'][i+1] > 1 : 
        posting_date = [fol_c['posting_date'][i] + datetime.timedelta(days=j+1) for j in range(int(fol_c['diff_days'][i+1]) - 1)]
        num_followers = [fol_c['num_followers'][i] * (1 + fol_c['growth_rate'][i+1])**(j+1) for j in range(int(fol_c['diff_days'][i+1]) - 1)]
        df = pd.DataFrame({'posting_date': posting_date, 'num_followers': num_followers})
        panda.append(df)
fol_new = pd.concat(panda, ignore_index=True)

prep = fol_c[['posting_date','num_followers']]

fol_end = pd.concat([prep,fol_new])
fol_end = fol_end.sort_values(by='posting_date', ascending=True,ignore_index=True)
days = (fol_end['posting_date'].iloc[-1] - fol_end['posting_date'].iloc[0])
days = days.days
growth_rate = (fol_end['num_followers'].iloc[-1]/fol_end['num_followers'].iloc[0])**(1/days) - 1
data_inst_n['posting_date'] = pd.to_datetime(data_inst_n['posting_date'])
data_inst_n = data_inst_n.sort_values(by='posting_date', ascending=True,ignore_index=True)
fol_end = fol_end.append(data_inst_n.iloc[0])
fol_end = fol_end.sort_values(by='posting_date', ascending=True,ignore_index=True)
days_delta = (fol_end['posting_date'].iloc[1] - fol_end['posting_date'].iloc[0])
days_delta = days_delta.days
if days_delta > 1 : 
    posting_date = [fol_end['posting_date'][1] - datetime.timedelta(days=j+1) for j in range(days_delta)]
    num_followers = [fol_end['num_followers'][1] / (1 + growth_rate)**(j+1) for j in range(days_delta)]
    df = pd.DataFrame({'posting_date': posting_date, 'num_followers': num_followers})
fol_final = pd.concat([fol_end,df],ignore_index=True)
fol_final = fol_final.sort_values(by='posting_date', ascending=True,ignore_index=True)
fol_final= fol_final.drop(fol_final.index[0]).reset_index()
final = data_inst_n.merge(fol_final, how='right', on='posting_date')
final = final.drop(columns=['num_followers_x','index'])
final.rename(columns = {'num_followers_y':'num_followers'}, inplace = True) 
final['influencer'] = 'Dua Lipa'
final = final.drop_duplicates(subset=['posting_date'])

In [5]:
gathered.append(final)
ended = pd.concat(gathered,ignore_index=True)

In [6]:
ended = ended.drop(columns=['level_0'])
ended

,posting_date,num_followers,influencer
0,2018-10-14,1.538133e+08,Cristiano Ronaldo
1,2018-10-16,1.540010e+08,Cristiano Ronaldo
2,2018-10-17,1.540949e+08,Cristiano Ronaldo
3,2018-10-18,1.541889e+08,Cristiano Ronaldo
4,2018-10-19,1.542830e+08,Cristiano Ronaldo
...,...,...,...
36719,2020-12-29,5.780829e+07,Dua Lipa
36720,2021-01-02,5.809246e+07,Dua Lipa
36721,2021-01-06,5.837801e+07,Dua Lipa
36722,2021-01-07,5.844962e+07,Dua Lipa


In [7]:
export = ended.to_csv (r'C:\Users\basti\DataCamp\Projet_Groupe\followers\followers_evolution.csv', index = None, header=True) 

### Merge with clean data

In [8]:
test = pd.read_csv(r'C:\Users\basti\DataCamp\Projet_Groupe_vclean\data\3_clean_data\data_inst.csv')
test = test.rename(columns = {'posting_date':'posting_date_complete'}, inplace = False) 

In [9]:
test['posting_date'] = test['posting_date_complete'].apply(date_inst)

In [10]:
ended['posting_date'] = pd.to_datetime(ended['posting_date'])
test['posting_date'] = pd.to_datetime(test['posting_date'])

In [11]:
test['posting_date'] = test['posting_date'].dt.date
ended['posting_date'] = ended['posting_date'].dt.date

In [12]:
dataframes=[]
for infl in test['influencer'].unique():
    set_1 = test[test['influencer']==infl]
    set_1 = set_1.reset_index()
    set_2 = ended[ended['influencer']==infl]
    set_2 = set_2.reset_index()
    num_followers_evolution = [set_2[set_2['posting_date']==set_1['posting_date'].iloc[i]]['num_followers'].values[0] for i in range(len(set_1))]
    set_1['num_followers_evolution']=num_followers_evolution
    dataframes.append(set_1)

In [13]:
done = pd.concat(dataframes)

In [14]:
done

,index,influencer,pr_activity,num_posts,num_followers,num_followings,post_description,num_comments,num_likes,posting_date_complete,media_type,year,month,day,hour,Day_week,posting_date,num_followers_evolution
0,0,Cristiano Ronaldo,Sportif,2982,253140888,459,Another week of playing inside and staying act...,22995,15023301,2020-04-11 09:02:26+00:00,Video,2020,4,11,9,Saturday,2020-04-11,2.144508e+08
1,1,Cristiano Ronaldo,Sportif,2982,253140888,459,We wish a Happy Easter to everyone 🌍❤️🙏\r\n#st...,62374,9661590,2020-04-12 13:18:16+00:00,Photo,2020,4,12,13,Sunday,2020-04-12,2.145816e+08
2,2,Cristiano Ronaldo,Sportif,2982,253140888,459,".\r\n\r\nKids, let the Dad do his work 🤷🏻‍♂️❤️...",108402,43462083,2020-04-13 12:57:43+00:00,Video,2020,4,13,12,Monday,2020-04-13,2.147125e+08
3,3,Cristiano Ronaldo,Sportif,2982,253140888,459,Feeling very thankful to have my mum home from...,62612,9435202,2020-03-28 23:17:59+00:00,Photo,2020,3,28,23,Saturday,2020-03-28,2.126278e+08
4,4,Cristiano Ronaldo,Sportif,2982,253140888,459,"In this difficult moment for the whole world, ...",67666,9668361,2020-03-30 12:58:03+00:00,Photo,2020,3,30,12,Monday,2020-03-30,2.128873e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,21532,Snoop Dogg,Musicien/Musicienne,47998,55395629,4313,Repost from @bigu1\n•\nMy New Workout...... Ya...,2640,1398627,2021-01-08 18:00:52+00:00,Video,2021,1,8,18,Friday,2021-01-08,5.536190e+07
488,21533,Snoop Dogg,Musicien/Musicienne,47998,55395629,4313,No caption,384,74540,2021-01-09 04:42:58+00:00,Photo,2021,1,9,4,Saturday,2021-01-09,5.537316e+07
489,21534,Snoop Dogg,Musicien/Musicienne,47998,55395629,4313,Repost from @isaachayes3\n•\nSleep * Restart *...,14921,4050094,2021-01-09 04:46:00+00:00,Video,2021,1,9,4,Saturday,2021-01-09,5.537316e+07
490,21535,Snoop Dogg,Musicien/Musicienne,47998,55395629,4313,🙏🏽😢,532,43367,2021-01-08 20:22:09+00:00,Photo,2021,1,8,20,Friday,2021-01-08,5.536190e+07


In [15]:
export = ended.to_csv (r'C:\Users\basti\DataCamp\Projet_Groupe\followers\clean_data.csv', index = None, header=True) 